# Routing

All necessary imports and openroutservices api key

In [12]:
import json
import folium
import requests
import mysql.connector
import pandas as pd
import openrouteservice as ors
from mysql.connector import Error
from IPython.display import Image
from openrouteservice.directions import directions

client = ors.Client(key='5b3ce3597851110001cf6248fadb94ffd01d455fb2bb86e68caf8f87')

Coordinates for loadingstation

In [13]:
# Filtering df according to needs
search = 'Lausanne'

df_loadingstations = pd.read_csv('../Ausgabedateien/locations_Ladestationen.csv')
df_filtered = df_loadingstations[df_loadingstations['City'] == search]

lats = df_filtered.iloc[1, 1]
lons = df_filtered.iloc[1, 2]

print(df_filtered)

      Unnamed: 0   Latitude  Longitude      City Country PostalCode  \
1448        1623  46.536810   6.617336  Lausanne     CHE       1018   
1449        1624  46.536810   6.617336  Lausanne     CHE       1018   
1876        2051  46.540969   6.658346  Lausanne     CHE       1010   
1877        2052  46.540969   6.658346  Lausanne     CHE       1010   
1878        2053  46.540969   6.658346  Lausanne     CHE       1010   

                        Street OperatorName  
1448  Route de Gratta-Paille 2   Bgdi_Admin  
1449  Route de Gratta-Paille 2   Bgdi_Admin  
1876        Route de Berne 144   Bgdi_Admin  
1877        Route de Berne 144   Bgdi_Admin  
1878        Route de Berne 144   Bgdi_Admin  


Coordinates for start address

In [14]:
# Set the API endpoint and your API key
endpoint = "https://api.openrouteservice.org/geocode/search"
api_key = "5b3ce3597851110001cf6248fadb94ffd01d455fb2bb86e68caf8f87"

# Prompt the user to enter a location
location = 'ZHAW School of Management and Law, St.-Georgen-Platz 2, 8401 Winterthur'

# Set the request parameters
params = {
    "api_key": api_key,
    "text": location
}

# Make the request
response = requests.get(endpoint, params=params)

# Check the status code of the response
if response.status_code == 200:
    # If the request was successful, get the coordinates from the response
    coordinates_start = response.json()["features"][0]["geometry"]["coordinates"]
    print(f"Coordinates of {location}: {coordinates_start}")
else:
    # If the request failed, print the error message
    print(response.json()["error"]["message"])


Coordinates of ZHAW School of Management and Law, St.-Georgen-Platz 2, 8401 Winterthur: [8.726028, 47.502264]


Coordinates for destination address

In [15]:
# Set the API endpoint and your API key
endpoint = "https://api.openrouteservice.org/geocode/search"
api_key = "5b3ce3597851110001cf6248fadb94ffd01d455fb2bb86e68caf8f87"

# Prompt the user to enter a location
location = 'Av. de Beau-Séjour 6, 1206 Genève'

# Set the request parameters
params = {
    "api_key": api_key,
    "text": location
}

# Make the request
response = requests.get(endpoint, params=params)

# Check the status code of the response
if response.status_code == 200:
    # If the request was successful, get the coordinates from the response
    coordinates_destination = response.json()["features"][0]["geometry"]["coordinates"]
    print(f"Coordinates of {location}: {coordinates_destination}")
else:
    # If the request failed, print the error message
    print(response.json()["error"]["message"])


Coordinates of Av. de Beau-Séjour 6, 1206 Genève: [6.149957, 46.1912]


Generating route

In [16]:
coords1 = coordinates_start
coords2 = lons, lats
coords3 = coordinates_destination

route1 = directions(client, ((coords1), (coords2), (coords3)),
                    format_out='geojson', profile='driving-hgv') # ORS

map_osm = folium.Map(location=[46.850860, 7.4316805], zoom_start=8) # Folium Map
folium.GeoJson(route1).add_to(map_osm) # Add to map

Processing the distance and time of the route

In [17]:
# Process data
time = 0 # Total time counter
distance = 0 # Total distance counter

for x in range(0, 1):
    coordinates = (coords1, coords2, coords3) # Coordinates
    geojson = directions(client, coordinates, format_out='geojson') # ORS
    for y in range(0, len(geojson['features'][0]['properties']['segments'])): # Features
        time += geojson['features'][0]['properties']['segments'][y]['duration']
        distance += geojson['features'][0]['properties']['segments'][y]['distance']
    folium.GeoJson(geojson).add_to(map_osm) # Add to map
    print('OK! {} | {}'.format(x, coordinates))

OK! 0 | ([8.726028, 47.502264], (6.617336, 46.53681), [6.149957, 46.1912])


In [18]:
distance = (round(distance/1000, 2))
time = (round(time/60/60, 2))

print('Total Length: {} km'.format(distance))
print('Total Time: {} h'.format(time))

Total Length: 315.53 km
Total Time: 3.59 h


Showing route on the map

In [1]:
map_osm

NameError: name 'map_osm' is not defined

# MySQL

Data to be safed on the Data Analytics database

In [20]:
# Test car Tesla Model X 75D / cosumption of 20.8 kWh/100km
usage = 20.8/100 # 1km
kWh= 0 # used
batteryused = 0 # Starting to drive
battery = 100 # Equals 75 kWh

# Optimal for fast charging
kWh = 60
battery = 20 # After 288.46 km

# Battery after drive 
kWh = (round(usage*distance/1, 2))
battery = (round((1.33*(75-usage*69.4))/1, 2))


print(f"{distance} km")
print(f"{time} h")
print(f"{battery} %")
print(f"{kWh} kWh")

315.53 km
3.59 h
80.55 %
65.63 kWh


Connecting to the MySQL server

In [21]:
cnx = mysql.connector.connect(user='root', password='George_George_MySQL', host='127.0.0.1', database='dataanalytics')

cursor= cnx.cursor() # created a cursor for the data

Executing the query

In [22]:
query = "INSERT INTO routes (distance, time, battery, kWh) VALUES (%s, %s, %s, %s)" #mysql query according to needs
values = (distance, time, battery, kWh) # values that should be safed

# Hinzufügen von Query um Daten von MySQL zu lesen

cursor.execute(query, values) # safing into cursor object
cnx.commit() # execute

cursor.close()
cnx.close()